## Content

1. Goal
2. Setup
3. Retrieve Data from Coins
4. Store Data

## Goal

The following data is retrieved through the API:
- Name of coin
- Price
- Market caps
- Total volumes
- Date

Coins for which data is pulled:
- Audius
- Ecomi
- FTX
- Convex Finance
- Dopex
- Rari Governance Token
- Spell Token
- Raydium
- GMX
- NFTX
- OCEAN
- Gro DAO Token
- Ribbon Finance

## Setup

In [1]:
# If you installed Python using Anaconda or Miniconda, then use conda:
# conda install ...

# If you installed Python any other way, then use pip:
# !pip install datetime

In [2]:
import sys
import requests
import pandas as pd
from datetime import datetime

In [3]:
# pd.options.display.max_rows = 4000
# pd.options.display.max_columns = 50

## Retrieve Data from Coins

Get historical market data include price, market cap, and 24h volume (granularity auto) \
Minutely data will be used for duration within 1 day, Hourly data will be used for duration between 1 day and 90 days, Daily data will be used for duration above 90 days.

In [4]:
# hourly data for the last 90days
coins = ["convex-finance", "ribbon-finance", "rari-governance-token", "gmx", "nftx", "ftx-token", "spell-token", "dopex" , "gro-dao-token", "audius", "ecomi", "raydium"]

column_names = ["prices", "market_caps", "total_volumes", "coin"]
df_coin_historical_data = pd.DataFrame(columns=column_names)

for coin in coins:
    url = 'https://api.coingecko.com/api/v3/coins/' + coin + '/market_chart?vs_currency=usd&days=90'
    r = requests.get(url)
    r.json()
    df_coin = pd.DataFrame(r.json())    
    df_coin["coin"] = coin
    df_coin_historical_data = df_coin_historical_data.append(df_coin, ignore_index=True)

for i, value in df_coin_historical_data.iterrows():
    df_coin_historical_data.loc[i, "date"] = datetime.utcfromtimestamp(df_coin_historical_data.loc[i,"prices"][0]/1000).strftime('%Y-%m-%d %H:%M:%S')
    df_coin_historical_data.loc[i, "date_unix"] = df_coin_historical_data.loc[i,"prices"][0]/1000 #timestamp is given in milliseconds
    df_coin_historical_data.loc[i, "prices"] = df_coin_historical_data.loc[i,"prices"][1]
    df_coin_historical_data.loc[i, "market_caps"] = df_coin_historical_data.loc[i,"market_caps"][1]
    df_coin_historical_data.loc[i, "total_volumes"] = df_coin_historical_data.loc[i,"total_volumes"][1]

df_coin_historical_data.head(3)

,prices,market_caps,total_volumes,coin,date,date_unix
0,19.5598,6.28498e+08,2.12461e+07,convex-finance,2021-10-23 06:37:36,1.634971e+09
1,19.5079,6.281e+08,2.03748e+07,convex-finance,2021-10-23 07:38:01,1.634975e+09
2,19.387,6.26893e+08,1.7392e+07,convex-finance,2021-10-23 08:37:23,1.634978e+09


## Store Data

In [5]:
df_coin_historical_data.to_csv("./Datasets/historical_data_coin.csv", index= False)